In [1]:
import numpy as np
import pandas as pd

from tqdm.notebook import tqdm

from airsim.collections import AirObject, AirEnv, RadarSystem, ControlPoint
from airsim.time import Time

%matplotlib inline

/var/folders/gv/4nk2tvfd4tjdhkr_hnd4tsz00000gn/T/ipykernel_86350/3714074986.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
def add_suffix(columns, suffix='_'):
    return [f'{col}{suffix}' for col in columns]

In [3]:
class DataGenerator:

    def __init__(self, shuffle_train=False, dropna_train=True):
        self.__shuffle_train = shuffle_train
        self.__dropna_train = dropna_train
        
        self.__t_min = None
        self.__t_max = None
        self.__dt = None

        self.__air_objects = None
        self.__air_env = None
        self.__radar_systems = None
        self.__control_point = None

    def __run_system(self, dropna=True):
        t = Time()
        t.set(self.__t_min)

        progressbar = tqdm(range(int((self.__t_max - self.__t_min)/self.__dt)))
        progressbar.set_description('Running system')
        for i in progressbar:
            for radar_system in self.__radar_systems:
                radar_system.trigger()

            if i == int((self.__t_max - self.__t_min)/self.__dt) - 1:
                for radar_system in self.__radar_systems:
                    radar_system.estimate_velocity()
                    radar_system.estimate_acceleration()
                self.__control_point.upload_data()
            
            t.step(self.__dt)

    def __get_control_point_data(self):
        data = self.__control_point.get_data()
        if self.__dropna_train:
            data = data.dropna()
        return data

    def __get_past_new_detections_cross_join_by_time(self, data, t):
        detection_columns = ['time', 'x', 'y', 'z', 'x_err', 'y_err', 'z_err']
        v_est_columns = [f'v_{axis}_est' for axis in ('x', 'y', 'z')]
        a_est_columns = [f'a_{axis}_est' for axis in ('x', 'y', 'z')]
    
        suffix = '_'
        cols = detection_columns + v_est_columns + a_est_columns
        cols_new = add_suffix(detection_columns + v_est_columns + a_est_columns, suffix)
        dfl = data.loc[data['time'] < t].copy()
        dfl = dfl.sort_values(by=['id', 'time', 'err_ratio'], ascending=[True, False, False])
        dfl = dfl.drop_duplicates(subset=['id', 'time'])
        dfl = dfl.groupby(by=['id']).head(1)
        dfl = dfl.rename(columns={cols[i]: cols_new[i] for i in range(len(cols))})
        dfl = dfl[['id'] + cols_new]
        dfl = dfl.reset_index(drop=True)
        
        dfr = data.loc[data['time'] == t].copy()
        dfr = dfr.sort_values(by=['id', 'err_ratio'], ascending=[True, False])
        dfr = dfr[['id'] + detection_columns]
        dfr = dfr.reset_index(drop=True)
    
        df = pd.merge(dfl, dfr, how='cross')
        df['is_identical'] = df['id_x'] == df['id_y']
        df = df.astype({'is_identical': 'float64'})
        df = df.drop(columns=['id_x', 'id_y'])
    
        return df

    def __get_past_new_detections_cross_join(self, data):
        dfs = []
        data['err_ratio'] = np.sqrt(3.0 / (data['x_err']**2 + data['y_err']**2 + data['z_err']**2))

        timestamps = sorted(set(data['time']))[1:]
        progressbar = tqdm(range(len(timestamps)))
        progressbar.set_description('Generating input data')
        for t_id in progressbar:
            df_t = self.__get_past_new_detections_cross_join_by_time(data, timestamps[t_id])
            dfs.append(df_t)
    
        df = pd.concat(dfs).reset_index(drop=True)
        if self.__shuffle_train:
            df = df.sample(frac=1).reset_index(drop=True)
        return df

    def generate_1ao_2rs(self, ao_track, t_min, t_max, dt):
        self.__t_min = t_min
        self.__t_max = t_max
        self.__dt = dt
        
        self.__air_objects = [AirObject(track=ao_track)]
        self.__air_env = AirEnv(air_objects=self.__air_objects)
        self.__radar_systems = [
            RadarSystem(position=np.array([0, 0, 0]), detection_radius=10**10, error=0.1, air_env=self.__air_env),
            RadarSystem(position=np.array([0, 0, 0]), detection_radius=10**10, error=0.1, air_env=self.__air_env)
        ]
        self.__control_point = ControlPoint(radar_systems=self.__radar_systems)

        self.__run_system()
        return self.__get_past_new_detections_cross_join(self.__get_control_point_data())

    def generate_Nao_2rs(self, ao_tracks, t_min, t_max, dt):
        self.__t_min = t_min
        self.__t_max = t_max
        self.__dt = dt
        
        self.__air_objects = [AirObject(track=ao_tracks[i]) for i in range(len(ao_tracks))]
        self.__air_env = AirEnv(air_objects=self.__air_objects)
        self.__radar_systems = [
            RadarSystem(position=np.array([0, 0, 0]), detection_radius=10**10, error=0.1, air_env=self.__air_env),
            RadarSystem(position=np.array([0, 0, 0]), detection_radius=10**10, error=0.1, air_env=self.__air_env)
        ]
        self.__control_point = ControlPoint(radar_systems=self.__radar_systems)

        self.__run_system()
        return self.__get_past_new_detections_cross_join(self.__get_control_point_data())


In [4]:
dg = DataGenerator()

In [8]:
def f(t):
    if t <= 15000:
        return np.array([t, 0, 10000])
    elif t <= 30000:
        return np.array([30000 - t, 0, 10000])
    elif t <= 45000:
        return np.array([0.5 * (t - 30000), 0, 10000])
    elif t <= 60000:
        return np.array([15000 - 0.5 * (t - 30000), 0, 10000])

df = dg.generate_1ao_2rs(ao_track=f, t_min=0, t_max=60000, dt=1)

df.to_csv('data/1ao_2rs_f.csv', index=False)

  0%|          | 0/60000 [00:00<?, ?it/s]

  0%|          | 0/59997 [00:00<?, ?it/s]

In [9]:
def g(t):
    return np.array([0.7 * t, 0, 10000])

df = dg.generate_1ao_2rs(ao_track=g, t_min=0, t_max=1000, dt=1)

df.to_csv('data/1ao_2rs_g.csv', index=False)

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/997 [00:00<?, ?it/s]

In [7]:
def h(t):
    return np.array([1.5 * t, 0, 10000])

df = dg.generate_1ao_2rs(ao_track=h, t_min=0, t_max=1000, dt=1)

df.to_csv('data/1ao_2rs_h.csv', index=False)

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/997 [00:00<?, ?it/s]